## Import

In [43]:
import json
import folium 
import requests
import pandas as pd
import shapely as shp
import geopandas as gpd
from copy import deepcopy
from sodapy import Socrata
import plotly.express as px

## Ingest data

### Open data Lombardia

#### Utility functions

In [41]:
def create_geometry(row, mappping):
    lat = row[mappping['wgs84_y']]
    lon = row[mappping['wgs84_x']]
    return shp.geometry.Point(lon, lat)

#### Load dataframe

In [3]:
client = Socrata("www.dati.lombardia.it",
                 "UoUBbv9A1VT5Fxv6LxYu6LlU2",
                 username="m.scatassi@campus.unimib.it",
                 password="wSau!K6Ad!mi4fU")

In [104]:
results = client.get("mtva-9hrb", 
                     where="comune='MILANO'")

In [105]:
df = pd.DataFrame.from_records(results)

In [106]:
df.head(5)

,id,istat,provincia,comune,indirizzo,civico,data_attiv,data_aut,sup_alim,sup_non_alim,sup_tot,wgs84_y,wgs84_x,location
0,480452,15146,MILANO,MILANO,Piazza PREALPI,7,2013-05-29T00:00:00.000,2013-05-29T00:00:00.000,0,37,37,9.15249709999714,45.49438969887889,"{'human_address': '{""address"": ""45 49438969887..."
1,469652,15146,MILANO,MILANO,PIAZZALE LORETO,SNC,1998-04-20T00:00:00.000,1998-04-20T00:00:00.000,190,0,190,9.21495479999597,45.486261598879004,"{'human_address': '{""address"": ""45 48626159887..."
2,469692,15146,MILANO,MILANO,Via ANSPERTO,7,1998-08-26T00:00:00.000,1998-08-26T00:00:00.000,0,90,90,9.178966699996648,45.46470579887949,"{'human_address': '{""address"": ""45 46470579887..."
3,489239,15146,MILANO,MILANO,VIA BASSI UGO,5,2021-11-05T00:00:00.000,2021-11-05T00:00:00.000,0,125,125,9.184472699996537,45.48905239887895,"{'human_address': '{""address"": ""45 48905239887..."
4,470278,15146,MILANO,MILANO,Via BELFIORE,9,1999-10-25T00:00:00.000,1999-10-25T00:00:00.000,0,39,39,9.15784739999704,45.467346698879496,"{'human_address': '{""address"": ""45 46734669887..."


The fields:
- 'data_attiv'
- 'data_aut'

are the same, so we can drop one of them.

In [90]:
(df.data_attiv == df.data_aut).all()

True

In [91]:
df.drop('data_aut', axis=1, inplace=True)

The fields:
- sup_alim
- sup_non_alim
- sup_tot

are strings, so we need to convert them to numbers.

In [115]:
df.sup_alim = df.sup_alim.apply(lambda x: float(x))
df.sup_non_alim = df.sup_non_alim.apply(lambda x: float(x))
df.sup_tot = df.sup_tot.apply(lambda x: float(x))

We are interested only in the alimentary stores, so we can drop the other rows.

In [116]:
df = df[df.sup_alim > 0]

#### Convert to gdf

In [117]:
mappping = {col: i for i, col in enumerate(df.columns)}

In [118]:
df["geometry"] = df.apply(create_geometry, mappping=mappping, axis=1)

In [127]:
gdf = gpd.GeoDataFrame(df, geometry="geometry")
gdf = gdf.set_crs(epsg=4326).reset_index(drop=True)

#### Visualize data

In [128]:
center = shp.convex_hull(shp.geometry.MultiPoint(gdf.geometry)).centroid.coords[0]

In [131]:
map = folium.Map(location=center, zoom_start=12)
for point, sup_tot in zip(gdf.geometry, gdf.sup_tot):
    if sup_tot > 250:
        folium.Circle([point.x, point.y], 
                      color="red",
                      radius=sup_tot*0.1).add_to(map)
    else:
        folium.Circle([point.x, point.y],
                      color="blue").add_to(map)
    
map

### Open Street Map

#### Utility functions

In [2]:
def extract_shop_tags(row, mapping):
    if type(row[mapping["other_tags"]])==str and '"shop"=>' in row[mapping["other_tags"]]:
        a = row[mapping["other_tags"]].split('"shop"=>')[1]
        if "," in a:
            a = a.split(",")[0]
        a = a.replace('"', '').replace(" ", "")
        return a
    else:
        return None

In [3]:
def extract_food_related_categories(row, mapping, food_related_categories):
    if type(row[mapping["shop"]])==str:
        if row[mapping["shop"]] in food_related_categories:
            return row[mapping["shop"]]
    return None

#### Code

In [4]:
# Read the GeoPackage file
data = gpd.read_file(r'C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\015146_Milano-2023-05-23T02Z.gpkg')

In [5]:
data.head(10)

,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags,geometry
0,13595397,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.13066 45.49952)
1,13919635,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,"""crossing""=>""marked""",POINT (9.16073 45.48770)
2,13919640,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.15375 45.48613)
3,13919655,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,"""crossing""=>""marked""",POINT (9.16219 45.48767)
4,13919747,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.16079 45.48079)
5,13919750,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,"""crossing""=>""traffic_signals""",POINT (9.16551 45.48758)
6,14461110,Milano Gallaratese,NaN,motorway_junction,2,NaN,NaN,NaN,NaN,NaN,POINT (9.06647 45.50117)
7,14461113,Settimo Milanese;Milano San Siro,NaN,motorway_junction,3,NaN,NaN,NaN,NaN,NaN,POINT (9.06639 45.49151)
8,14461355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""comment""=>""connection toward Venezia closed f...",POINT (9.06496 45.50572)
9,14461396,Settimo Milanese,NaN,motorway_junction,3a,NaN,NaN,NaN,NaN,NaN,POINT (9.06699 45.48894)


In [6]:
mapping = {col: i for i, col in enumerate(data.columns)}

In [7]:
data["shop"] = data.apply(lambda x: extract_shop_tags(x, mapping), axis=1)

In [8]:
data.head(5)

,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags,geometry,shop
0,13595397,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.13066 45.49952),None
1,13919635,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,"""crossing""=>""marked""",POINT (9.16073 45.48770),None
2,13919640,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.15375 45.48613),None
3,13919655,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,"""crossing""=>""marked""",POINT (9.16219 45.48767),None
4,13919747,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.16079 45.48079),None


In OpenStreetMap, the "shop" tag is used to describe various types of commercial establishments. When it comes to food-related categories for the "shop" tag, there are several options commonly used to represent different types of food-related businesses.

In [24]:
food_categories = [
    "alcohol",
    "bakery",
    "beverages",
    "brewing_supplies",
    "butcher",
    "cheese",
    "chocolate",
    "coffee",
    "confectionery",
    "convenience",
    "deli",
    "dairy",
    "farm",
    "frozen_food",
    "greengrocer",
    "health_food",
    "ice_cream",
    "pasta",
    "pastry",
    "seafood",
    "spices",
    "tea",
    "wine",
    "water",
    "supermarket",
    "food"
]

Remove non-food-related shops.

In [25]:
mapping = {col: i for i, col in enumerate(data.columns)}

In [26]:
food_related_rows = data.apply(lambda x: extract_food_related_categories(x, mapping, food_categories), axis=1).dropna().index

In [27]:
food_data = data.iloc[food_related_rows]

Remove nan columns.

In [28]:
for col in food_data.columns:
    if food_data[col].isna().all():
        food_data.drop(col, axis=1, inplace=True)

C:\Users\Marco\AppData\Local\Temp\ipykernel_11428\4066126692.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Marco\AppData\Local\Temp\ipykernel_11428\4066126692.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Marco\AppData\Local\Temp\ipykernel_11428\4066126692.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Marco\AppData\Local\Temp\ipykernel_11428\4066126692.py:3: SettingWithCopyWarning:


A value

In [29]:
food_data.head(10)

,osm_id,name,ref,other_tags,geometry,shop
970,245055800,Pane e Dolci,NaN,"""shop""=>""bakery""",POINT (9.15500 45.46046),bakery
1137,250969620,Lidl,NaN,"""brand""=>""Lidl"",""brand:wikidata""=>""Q151954"",""b...",POINT (9.14200 45.44759),supermarket
1279,255137455,U2,NaN,"""old_name""=>""Unes"",""operator""=>""Unes"",""shop""=>...",POINT (9.24180 45.48799),supermarket
1301,255508749,Iper Milano Portello,NaN,"""addr:city""=>""Milano"",""addr:country""=>""IT"",""ad...",POINT (9.14540 45.49014),supermarket
1383,258035187,Pam,NaN,"""addr:housenumber""=>""6"",""addr:street""=>""Viale ...",POINT (9.19804 45.45080),supermarket
1397,258076731,Carrefour Market,NaN,"""addr:city""=>""Milano"",""addr:housenumber""=>""24/...",POINT (9.16623 45.48802),supermarket
1398,258076763,Pam local,NaN,"""addr:city""=>""Milano"",""addr:housenumber""=>""8"",...",POINT (9.16506 45.48880),supermarket
1443,259592419,Lidl,NaN,"""brand""=>""Lidl"",""brand:wikidata""=>""Q151954"",""b...",POINT (9.14885 45.46232),supermarket
1635,266384203,Punto SMA,NaN,"""addr:housenumber""=>""38"",""addr:postcode""=>""201...",POINT (9.20268 45.45128),supermarket
1701,268403462,Mercato Comunale,NaN,"""shop""=>""convenience"",""wheelchair""=>""yes""",POINT (9.16267 45.43128),convenience


### Visualize data

#### Scatter plot

In [30]:
fig = px.scatter_mapbox(food_data, 
                        lat=food_data.geometry.y, 
                        lon=food_data.geometry.x,
                        color="shop", 
                        hover_name="name",
                        zoom=10)

fig.update_layout(title="<b>Food store across Milan<b>",
                  mapbox_style="open-street-map",
                  title_pad_l=350,
                  height=700,
                  width=1100,
                  xaxis_title="time of the day",)

fig.update_traces(cluster=dict(enabled=True))

#### Chorepleth map 

##### Density by area

In [31]:
NIL = gpd.read_file(r'C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\NIL_geometry.geojson')

In [32]:
NIL

,FID_1,FID_1_1,ID_NIL,NIL,AreaHA,AreaMQ,geometry
0,0,0,74,SACCO,70.84658,7.084658e+05,"POLYGON ((9.12195 45.51602, 9.12163 45.51589, ..."
1,1,1,82,COMASINA,92.67346,9.267346e+05,"POLYGON ((9.16887 45.52397, 9.16803 45.52234, ..."
2,2,2,75,STEPHENSON,56.00979,5.600979e+05,"POLYGON ((9.12933 45.50998, 9.12973 45.50939, ..."
3,3,3,66,QT 8,102.44374,1.024437e+06,"POLYGON ((9.14368 45.48474, 9.14338 45.48420, ..."
4,4,4,29,ORTOMERCATO,140.25196,1.402520e+06,"POLYGON ((9.23739 45.45588, 9.23731 45.45427, ..."
...,...,...,...,...,...,...,...
83,83,83,6,TICINESE,125.50647,1.255065e+06,"POLYGON ((9.18675 45.45235, 9.18659 45.45183, ..."
84,84,84,47,CANTALUPA,92.67168,9.267168e+05,"POLYGON ((9.15445 45.41758, 9.15433 45.41743, ..."
85,85,85,86,PARCO DEI NAVIGLI,361.78363,3.617836e+06,"POLYGON ((9.15266 45.41520, 9.15200 45.41635, ..."
86,86,86,68,PAGANO,128.97343,1.289734e+06,"POLYGON ((9.16506 45.46684, 9.16486 45.46619, ..."


In [33]:
NIL["food_count"] = [0]*len(NIL)

for i in range(len(NIL)):
    for j in range(len(food_data)):
        if NIL.iloc[i].geometry.contains(food_data.iloc[j].geometry):
            NIL.at[i, "food_count"] = NIL.iloc[i]["food_count"] + 1

In [34]:
NIL["food_density"] = NIL["food_count"]/NIL["AreaHA"]

In [35]:
NIL

,FID_1,FID_1_1,ID_NIL,NIL,AreaHA,AreaMQ,geometry,food_count,food_density
0,0,0,74,SACCO,70.84658,7.084658e+05,"POLYGON ((9.12195 45.51602, 9.12163 45.51589, ...",1,0.014115
1,1,1,82,COMASINA,92.67346,9.267346e+05,"POLYGON ((9.16887 45.52397, 9.16803 45.52234, ...",3,0.032372
2,2,2,75,STEPHENSON,56.00979,5.600979e+05,"POLYGON ((9.12933 45.50998, 9.12973 45.50939, ...",0,0.000000
3,3,3,66,QT 8,102.44374,1.024437e+06,"POLYGON ((9.14368 45.48474, 9.14338 45.48420, ...",0,0.000000
4,4,4,29,ORTOMERCATO,140.25196,1.402520e+06,"POLYGON ((9.23739 45.45588, 9.23731 45.45427, ...",5,0.035650
...,...,...,...,...,...,...,...,...,...
83,83,83,6,TICINESE,125.50647,1.255065e+06,"POLYGON ((9.18675 45.45235, 9.18659 45.45183, ...",31,0.246999
84,84,84,47,CANTALUPA,92.67168,9.267168e+05,"POLYGON ((9.15445 45.41758, 9.15433 45.41743, ...",0,0.000000
85,85,85,86,PARCO DEI NAVIGLI,361.78363,3.617836e+06,"POLYGON ((9.15266 45.41520, 9.15200 45.41635, ...",0,0.000000
86,86,86,68,PAGANO,128.97343,1.289734e+06,"POLYGON ((9.16506 45.46684, 9.16486 45.46619, ...",18,0.139564


In [39]:
fig = px.choropleth(NIL, 
                    geojson=NIL.geometry.__geo_interface__,
                    locations=NIL.index,
                    color="food_density",
                    hover_data=["NIL", "food_count"],
                    color_continuous_scale='Viridis',
                    center = {"lat": 45.464664, "lon": 9.188540})

fig.update_layout(title="<b>Food store density (per NIL area) across Milan<b>",
                  mapbox_style="open-street-map",
                  title_pad_l=200,
                  height=700,
                  width=1100,)

fig.update_geos(fitbounds="locations", visible=False)

##### Density by population